### stats

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

url = "https://www.fotmob.com/matches/chelsea-vs-nottingham-forest/2vn3ne#4506599:tab=stats"

# Setup browser
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Remove this line if you want to see the browser
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

print("⏳ Loading page...")
driver.get(url)
time.sleep(5)

print("✅ Page loaded.")
html = driver.page_source
print("HTML length:", len(html))

driver.quit()


⏳ Loading page...
✅ Page loaded.
HTML length: 776460


In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

main_content = soup.find('div', id='__next')
if main_content:
    print("✅ Found #main-content")
else:
    print("❌ Couldn't find #main-content")


✅ Found #main-content


In [118]:
soup

<html data-color-scheme="dark" lang="en"><head><meta content="app-id=488575683" name="apple-itunes-app"/><link href="android-app://com.mobilefootie.wc2010/http" rel="alternate"/><link href="/img/android-icon-192x192.png" rel="apple-touch-icon"/><meta content="all" name="robots"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0" name="viewport"/><meta content="FotMob" property="og:site_name"/><meta content="204905456199565" property="fb:app_id"/><meta content="christer.nordvik" property="fb:admins"/><meta content="https://www.facebook.com/fotmob" property="article:author"/><meta content="https://www.facebook.com/fotmob" property="article:publisher"/><meta content="iAsqa4atH8Kb02MqS9Sc0oZC__7WrvJYU6zYqJqE6Ww" name="google-site-verification"/><link href="https://www.fotmob.com/topnews/feed" rel="alternate" title="Top news" type="application/atom+xml"/><link href="https://www.fotmob.com/matches/chelsea-vs-nottingham-forest/2vn3ne" hreflang="en"

In [24]:
match_css = main_content.find('div', class_='css-19auws2-MatchCSS edsvb150') if main_content else None
print("Found match_css:", bool(match_css))

inner_div = match_css.find('div') if match_css else None
left_column = inner_div.find('div', class_='css-10wb1x-Column-LeftColumn edsvb152') if inner_div else None

if left_column:
    print("✅ Found left_column")
else:
    print("❌ Could not find left_column")


Found match_css: True
✅ Found left_column


In [41]:
stats_section = left_column.find_all('section')
if not stats_section:
    print("❌ Stats section not found.")
    exit()

print("✅ Stats section found.")

✅ Stats section found.


In [42]:
stats_section

[<section><h1 style="position:absolute;left:-9999px">Nottingham Forest vs Chelsea (2025-05-25T15:00:00.000Z)</h1><div class="css-1g9gm8m-MFHeaderFullscreenCSS e1vkkyp0"><div class="css-1q3r0n9-MFHeaderTopBarMobileCSS e1pkr3n46"><div class="css-761wfz-MFHeaderButtonsCSS e1q00yv90"><button class="css-11ez6d8-FollowButtonCSS e45uhi50">Follow</button></div></div><div class="css-1jb4fga-HeaderTopBarDesktopCSS e1pkr3n45"><div class="css-1711rkn-GridContainer e1pkr3n40"><div class="css-ozxjdo-LeftGridItem e1pkr3n41"><button class="css-1600hk9-BackButton e1pkr3n44"><svg height="25px" version="1.1" viewbox="0 0 25 25" width="25px" xmlns="http://www.w3.org/2000/svg"><g fill="none" fill-rule="evenodd" id="Page-1" stroke="none" stroke-width="1"><g id="calendar-navigation"><circle cx="12.5" cy="12.5" fill="var(--DatePicker-DatepickerArrowButton-background)" fill-rule="nonzero" id="prefix__Ellipse_377" r="12.5"></circle><g id="prefix__ic_chevron_left" transform="translate(5.000000, 5.000000)"><polyg

In [124]:
soup.find_all('div', class_=lambda x: x and 'Score' in x)

[]

In [126]:
def scrape_match_details(soup):
    """
    Scrapes match details including date, time, teams, venue, and competition info
    """
    match_info = {}
    
    # Find header section
    header = soup.find('div', class_='css-1pf15hj-MFHeaderInfoBoxCSS')
    if not header:
        return match_info
    
    # Get date and time
    date_element = header.find('time')
    if date_element:
        match_info['datetime'] = date_element.get('datetime')
        match_info['formatted_date'] = date_element.text.strip()
    
    # Get competition info
    competition = header.find('div', class_='css-6k5lms-TournamentCSS')
    if competition:
        league_link = competition.find('a')
        if league_link:
            match_info['competition'] = {
                'name': league_link.text.strip(),
                'url': league_link.get('href'),
                # 'logo': competition.find('img')['src'] if competition.find('img') else None
            }
    
    
    # Get teams info (from the main content)
    
    # Get teams info from team markup elements
    team_sections = soup.find_all('div', class_=lambda x: x and 'TeamMarkup' in x)
    if len(team_sections) == 2:
        # Get team names (using desktop version of names)
        home_name = team_sections[0].find('span', class_=lambda x: x and 'TeamNameOnTabletUp' in x)
        away_name = team_sections[1].find('span', class_=lambda x: x and 'TeamNameOnTabletUp' in x)
        
        # Get team links and images
        home_link = team_sections[0].find_parent('a')
        away_link = team_sections[1].find_parent('a')
        
        home_img = team_sections[0].find('img')
        away_img = team_sections[1].find('img')
        
        match_info['teams'] = {
            'home': {
                'name': home_name.text.strip() if home_name else None,
                # 'url': home_link.get('href') if home_link else None,
                # 'logo': home_img.get('src') if home_img else None
            },
            'away': {
                'name': away_name.text.strip() if away_name else None,
                # 'url': away_link.get('href') if away_link else None,
                # 'logo': away_img.get('src') if away_img else None
            }
        }
        
        # Get score
        score_wrapper = soup.find('div', class_='css-1cf82ng-MFHeaderStatusWrapper')
        if score_wrapper:
            score_element = score_wrapper.find('span', class_=lambda x: x and 'Score' in x)
            if score_element:
                scores = score_element.text.strip().split(' - ')
                match_info['score'] = {
                    'home': scores[0] if len(scores) > 0 else None,
                    'away': scores[1] if len(scores) > 1 else None
                }
            
            # Get match status
            status_element = score_wrapper.find('span', class_=lambda x: x and 'StatusReason' in x)
            if status_element:
                match_info['status'] = status_element.text.strip()
    
    
    return match_info

# Usage:
match_details = scrape_match_details(soup)
print(json.dumps(match_details, indent=2))

{
  "datetime": "2025-05-25T15:00:00.000Z",
  "formatted_date": "Sun, May 25, 10:00 AM",
  "competition": {
    "name": "Premier League Round 38",
    "url": "/leagues/47/overview/premier-league"
  },
  "teams": {
    "home": {
      "name": "Nottingham Forest"
    },
    "away": {
      "name": "Chelsea"
    }
  },
  "score": {
    "home": "0",
    "away": "1"
  },
  "status": "Full time"
}


In [80]:
lf = left_column.find_all("li", class_=lambda c: c and "Stat" in c)

In [115]:
def scrape_stats(left_column):
    results = []

    # Correct class for stat blocks
    stat_blocks = left_column.find_all("li", class_=lambda c: c and "Stat" in c) # List item containing stats
    # print(f"{len(stat_blocks)} stat blocks found.")

    for i, block in enumerate(stat_blocks, 1):
        # print(f"\n--- Stat Block #{i} ---")
        # print(block.prettify())

        # Stat title comes from the <span> with class containing "StatTitle"
        title_tag = block.find("span", class_=lambda c: c and "StatTitle" in c)
        stat_title = title_tag.text.strip() if title_tag else None

        # Find all values (they’re wrapped in <span class="StatValue"> inside StatBox)
        value_tags = block.find_all("span", class_=lambda c: c and "StatValue" in c)
        home_value = value_tags[0].text.strip() if len(value_tags) > 0 else None
        away_value = value_tags[1].text.strip() if len(value_tags) > 1 else None

        # print(f"Extracted: {home_value} {stat_title} {away_value}")
        if home_value is not None or away_value is not None:
            results.append({
                stat_title: {
                    "home": home_value,
                    "away": away_value
                }
            })

    return results


In [95]:
import json
# Extract stats (assuming left_column is already defined)
stats = scrape_stats(left_column)

# Save to JSON
with open("match_stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)

# Print preview
# print(json.dumps(stats, indent=2, ensure_ascii=False))


46 stat blocks found.


In [117]:
print(json.dumps(scrape_stats(soup), indent=2))

[
  {
    "Expected goals (xG)": {
      "home": "1.20",
      "away": "1.09"
    }
  },
  {
    "Total shots": {
      "home": "10",
      "away": "6"
    }
  },
  {
    "Shots on target": {
      "home": "2",
      "away": "2"
    }
  },
  {
    "Big chances": {
      "home": "2",
      "away": "3"
    }
  },
  {
    "Big chances missed": {
      "home": "2",
      "away": "2"
    }
  },
  {
    "Accurate passes": {
      "home": "352 (84%)",
      "away": "327 (80%)"
    }
  },
  {
    "Fouls committed": {
      "home": "10",
      "away": "11"
    }
  },
  {
    "Corners": {
      "home": "7",
      "away": "4"
    }
  },
  {
    "Total shots": {
      "home": "10",
      "away": "6"
    }
  },
  {
    "Shots off target": {
      "home": "4",
      "away": "2"
    }
  },
  {
    "Shots on target": {
      "home": "2",
      "away": "2"
    }
  },
  {
    "Blocked shots": {
      "home": "4",
      "away": "2"
    }
  },
  {
    "Hit woodwork": {
      "home": "0",
      "away": "0"

### test

In [132]:
import json
from bs4 import BeautifulSoup
from get_match_stats import scrape_full_match_data

In [ ]:
# Get all match data using the scrape_full_match_data function
match_data = scrape_full_match_data(soup)

# Save complete match data to JSON
with open("full_match_data.json", "w", encoding="utf-8") as f:
    json.dump(match_data, f, indent=2, ensure_ascii=False)

print("✅ Data saved to full_match_data.json")


In [129]:
print(json.dumps(scrape_full_match_data(soup), indent=2, ensure_ascii=False))

{
  "match_details": {
    "teams": {
      "home": {
        "name": "Nottingham Forest",
        "url": "/teams/10203/overview/nottingham-forest",
        "logo": "https://images.fotmob.com/image_resources/logo/teamlogo/10203_small.png"
      },
      "away": {
        "name": "Chelsea",
        "url": "/teams/8455/overview/chelsea",
        "logo": "https://images.fotmob.com/image_resources/logo/teamlogo/8455_small.png"
      }
    },
    "score": {
      "home": "0",
      "away": "1"
    },
    "status": "Full time",
    "datetime": "2025-05-25T15:00:00.000Z",
    "formatted_date": "Sun, May 25, 10:00 AM",
    "competition": {
      "name": "Premier League Round 38",
      "url": "/leagues/47/overview/premier-league"
    }
  },
  "statistics": [
    {
      "Expected goals (xG)": {
        "home": "1.20",
        "away": "1.09"
      }
    },
    {
      "Total shots": {
        "home": "10",
        "away": "6"
      }
    },
    {
      "Shots on target": {
        "home": "2",
